# Previsão de Séries Temporais usando MLForecast

Processo construído de acordo com artigo técnico do Medium encontrado no link: https://mariofilho.com/como-prever-series-temporais-com-scikit-learn/

Os dados estão disponíveis no kaggle: https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data?select=transactions.csv.

### Sobre os dados:

Descrição dos Arquivos e Informações dos Campos de Dados
- **train.csv**:
Os dados de treinamento, consistindo em séries temporais com as características store_nbr, family e onpromotion, além do alvo sales.

    - `store_nbr`: Identifica a loja onde os produtos são vendidos.
    - `family`: Identifica o tipo de produto vendido.
    - `sales`: Fornece as vendas totais de uma família de produtos em uma loja específica em uma data determinada. Valores fracionários são possíveis, já que os produtos podem ser vendidos em unidades fracionárias (1,5 kg de queijo, por exemplo, em vez de 1 pacote de batatas fritas).
    - `onpromotion`: Indica o número total de itens de uma família de produtos que estavam em promoção em uma loja em uma data específica.

- **test.csv**:
Os dados de teste, com as mesmas características dos dados de treinamento. Você deverá prever o alvo sales para as datas presentes neste arquivo.

As datas nos dados de teste correspondem aos 15 dias após a última data nos dados de treinamento.

- **stores.csv**:
Metadados das lojas, incluindo cidade, estado, tipo e cluster.

    - `cluster`: Agrupamento de lojas similares.

- **oil.csv**:
Preço diário do petróleo. Inclui valores tanto no período dos dados de treinamento quanto nos dados de teste.

(O Equador é um país dependente do petróleo, e sua saúde econômica é altamente vulnerável a choques nos preços do petróleo.)

- **holidays_events.csv**
Feriados e eventos, com metadados.
NOTA: Preste atenção especial à coluna transferred. Um feriado transferido oficialmente ocorre naquele dia do calendário, mas foi movido para outra data pelo governo. Um dia transferido é mais semelhante a um dia normal do que a um feriado. Para encontrar o dia em que ele foi efetivamente celebrado, procure a linha correspondente onde o tipo é Transfer. Por exemplo, o feriado Independencia de Guayaquil foi transferido de 2012-10-09 para 2012-10-12, o que significa que foi celebrado em 2012-10-12.

Dias do tipo Bridge são dias extras adicionados a um feriado (por exemplo, para estender a pausa durante um final de semana prolongado). Esses dias são frequentemente compensados pelo tipo Work Day, que é um dia normalmente não agendado para trabalho (por exemplo, sábado), mas que é usado para compensar o Bridge.
Additional holidays são dias adicionais a um feriado regular do calendário, como geralmente ocorre ao redor do Natal (fazendo, por exemplo, a véspera de Natal ser um feriado).

---
**Notas Adicionais**
Salários no setor público são pagos quinzenalmente, no dia 15 e no último dia do mês. As vendas de supermercados podem ser afetadas por isso.
Um terremoto de magnitude 7.8 atingiu o Equador em 16 de abril de 2016. A população se mobilizou em esforços de ajuda, doando água e outros produtos de primeira necessidade, o que afetou significativamente as vendas dos supermercados por várias semanas após o terremoto.

Vamos observar dos dados de treino como estão nossas colunas:

In [9]:
# importanto bibliotecas
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [10]:
# leitura dos dados de treino

path = 'data/train.csv'
data = pd.read_csv(path, index_col= 'id', parse_dates=['date'])

In [11]:
data.head()

,date,store_nbr,family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,2013-01-01,1,BABY CARE,0.0,0
2,2013-01-01,1,BEAUTY,0.0,0
3,2013-01-01,1,BEVERAGES,0.0,0
4,2013-01-01,1,BOOKS,0.0,0


O grupo NixTla (criadores da `mlforecast`) padronizam o nome das colunas, para que o modelo consiga identificar. Assim temos as colunas:

- `ds`: será a coluna com a data;
- `y`: a nossa variável alvo (target)
- `unique_id`: a coluna que identifica cada série temporal que você tem.

No nosso caso, como as lojas são divididas por famílias, de acordo com a similaridade, então cada família representa uma série temporal. Logo nossa coluna `family`será renomeada para `unique_id`. 

Como temos várias lojas, temos de combinar os valores do family e o numero da loja, na coluna `unique_id`.

In [15]:
# renomeando as colunas
data2 = data.rename(columns= {'date': 'ds', 'sales': 'y'})
data2['unique_id'] = data2['store_nbr'].astype(str) + '_' + data2['family']


In [14]:
data2 = data2[['ds', 'unique_id', 'y', 'onpromotion']]
data2

,ds,unique_id,y,onpromotion
id,,,,
0,2013-01-01,1_AUTOMOTIVE,0.000,0
1,2013-01-01,1_BABY CARE,0.000,0
2,2013-01-01,1_BEAUTY,0.000,0
3,2013-01-01,1_BEVERAGES,0.000,0
4,2013-01-01,1_BOOKS,0.000,0
...,...,...,...,...
3000883,2017-08-15,9_POULTRY,438.133,0
3000884,2017-08-15,9_PREPARED FOODS,154.553,1
3000885,2017-08-15,9_PRODUCE,2419.729,148


Uma linha para cada registro contendo a data, o ID da série temporal (categoria do produto em nosso exemplo), o valor da variável alvo e o valor de variáveis externas.

Veja que as séries temporais estão empilhadas.

### Validação temporal

**O Problema: Vazamento de Dados** 

Quando trabalhamos com séries temporais, os dados possuem uma característica fundamental: dependem do tempo. Isso significa que os valores de uma data futura não podem ser conhecidos antes de essa data acontecer. Se você usar dados futuros (mesmo que indiretamente) para treinar o modelo, estará introduzindo o que chamamos de `vazamento de dados`.

**Por que a validação aleatória ou k-fold causa vazamento de dados?** 

- **Validação aleatória:**  Divide os dados de forma aleatória entre treino e validação. Isso significa que um ponto de dados de uma data futura pode acabar no conjunto de treino, enquanto os dados de uma data anterior podem ser usados na validação.
    - Exemplo: Um dado de 2023-12-01 no treino, e outro de 2023-11-01 na validação.
    - Problema: O modelo "aprendeu" com informações do futuro (2023-12-01), o que nunca aconteceria em um cenário real.
- **Validação k-fold:** Divide os dados em k subconjuntos (folds) e usa diferentes combinações para treino e validação. No caso de séries temporais, isso também pode misturar dados de diferentes períodos de tempo.
    - Problema: Como em k-fold os dados são reorganizados, informações futuras podem influenciar o treino de maneira não realista.
    
**O Impacto na Prática**
Na prática, quando você faz previsões, os dados do futuro não estarão disponíveis. Se o modelo treinou com esses dados, ele terá uma visão "irreal" das relações entre as variáveis, resultando em um desempenho artificialmente alto durante a validação. Esse desempenho não se sustentará em novos dados reais.

Vamos usar uma validação temporal simples entre passado e futuro!

In [20]:
# escolhendo a janela temporal

# Obtendo a data mínima e máxima
min_date = data2['ds'].min()
max_date = data2['ds'].max()

print("Data mínima:", min_date)
print("Data máxima:", max_date)

Data mínima: 2013-01-01 00:00:00
Data máxima: 2017-08-15 00:00:00


Vamos dividir os dados de treino e teste usando a seguinte estratégia:

- `dados de treino`: De 2013-01-01 até 2016-12-31
- `dados de teste`: De 2017-01-01 até 2017-08-15

In [21]:
# split dos dados

train  = data2.loc[data2.ds < '2017-01-01']
valid = data2.loc[data2.ds >= '2017-01-01']

### Featuring Engineering para Séries Temporais

Essa parte é crucial para o sucesso de seu modelo.

É de costume pensar em 4 tipos de features principais para séries temporais:

- **LAGS**: são simplesmente valores passados da série temporal que você inclui como features para o modelo.

    - Por exemplo, se você está tentando prever a demanda de um produto daqui a uma semana, pode incluir a demanda do mesmo dia da semana, mas na semana anterior como feature.
    - É importante testar vários valores de lags para encontrar o que funciona melhor para o seu problema específico.

- **AGREGAÇÕES TEMPORAIS**: As agregações temporais são operações aplicadas a uma janela de observações da série temporal.

    - Por exemplo, calcular a média de vendas diárias dos últimos 7 dias, ou o desvio padrão dos retornos de um ativo financeiro nas últimas 4 semanas.
    Essas agregações podem ajudar o modelo a identificar tendências e padrões na série temporal.

- **COMPONENTES DA DATA**: Os componentes da data são valores que você extrai de uma data ou hora específica.

    - Por exemplo, você pode incluir a hora do dia, o dia da semana, o mês, a estação do ano, etc.

    Essas features podem ajudar o modelo a identificar padrões sazonais e cíclicos na série temporal.

- **DIFERENÇAS**: As diferenças são o resultado da subtração entre valores da série temporal.

    - Por exemplo, você pode incluir a diferença entre a demanda de hoje e a demanda de ontem.

    Mesmo com os lags, as diferenças podem trazer informações mais detalhadas para o modeo e ajudá-lo a identificar os padrões com mais facilidade.


Vamos utilizar apenas 2 modelos para testar na nossa série.

In [22]:
# criando lista com modelos que queremos testar

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

models = [RandomForestRegressor(random_state=0, n_estimators=100),
          ExtraTreesRegressor(random_state=0, n_estimators=100)]

**EXPLICAÇÃO PARA OS PARAMETROS**
1. `random_state`
- O que é: Define a semente para o gerador de números aleatórios usado pelo modelo.

**Por que usar:**

Garante que os resultados sejam reprodutíveis.
Modelos como Random Forest e Extra Trees usam aleatoriedade em várias etapas, como:
        - Seleção de amostras para cada árvore.
        - Seleção de subconjuntos de recursos (features) ao construir as árvores.
Configurando random_state, o comportamento aleatório se torna determinístico, garantindo que o mesmo código sempre produza os mesmos resultados.

**Como definir:**

Escolha um número inteiro, por exemplo, random_state=0.
O número em si não importa, desde que você use o mesmo número para replicar resultados.

2. `n_estimators`
O que é: Número de árvores a serem criadas na floresta.

**Por que usar:**

- Aumentar n_estimators tende a melhorar a precisão do modelo (até certo ponto), porque mais árvores significam mais combinações de decisões e, geralmente, menor variância.
- Contudo, há um custo computacional: mais árvores aumentam o tempo de treinamento e inferência.

**Como definir:**

- Comece com um valor padrão, como n_estimators=100.
- Aumente para 200, 500 ou mais, caso seu modelo ainda não esteja performando bem e você tenha capacidade computacional suficiente.
- Use validação cruzada para testar diferentes valores e encontrar o equilíbrio entre desempenho e custo.

Para calcular a diferença entre os valores da série temporal, vamos utilizar a biblioteca `numba` que compila a função criada e otimiza a performance do código.

In [25]:
def diff(x, lag):
    x2 = np.full_like(x, np.nan)
    for i in range(lag, len(x)):
        x2[i] = x[i] - x[i-lag]
    return x2